### Test metrics on validation set (part of the official training set)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, multilabel_confusion_matrix

# Custom objects required for loading CheXpert multi-label classification models
import sys 
sys.path.append('..')
from loss import (
    set_binary_crossentropy_weighted_loss
)

### 1- Loading test set

In [ ]:
test_df = pd.read_csv(
    filepath_or_buffer="../labels/train_validation_split_data/validation_u-zeroes.csv",
    dtype={  # Setting labels to type np.float32 was necessary for conversion to tf.Tensor object
        "Path": str,
        "Atelectasis": np.float32,
        "Cardiomegaly": np.float32,
        "Consolidation": np.float32,
        "Edema": np.float32,
        "Pleural Effusion": np.float32,
        "Pleural Other": np.float32,
        "Pneumonia": np.float32,
        "Pneumothorax": np.float32,
        "Enlarged Cardiomediastinum": np.float32,
        "Lung Opacity": np.float32,
        "Lung Lesion": np.float32,
        "Fracture": np.float32,
        "Support Devices": np.float32,
        "No Finding": np.float32
    }
)

In [ ]:
list_columns = list(test_df.columns)
y_cols = list_columns[1::]  # First column is 'Path' column

In [ ]:
list_columns

In [ ]:
y_cols

In [ ]:
test_datagen = ImageDataGenerator(
    featurewise_center=True,  # Mean and standard deviation values of the training set will be loaded to the object
    featurewise_std_normalization=True
)

In [ ]:
training_dataset_mean = np.load("../misc/calculated_chexpert_training_dataset_mean_and_std_values/CheXpert_training_set_mean.npy")
training_dataset_std = np.load("../misc/calculated_chexpert_training_dataset_mean_and_std_values/CheXpert_training_set_std.npy")

In [ ]:
test_datagen.mean = training_dataset_mean
test_datagen.std = training_dataset_std

test_datagenerator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory="../dataset/",
    x_col='Path',
    y_col=y_cols,
    weight_col=None,
    target_size=(512, 512),
    color_mode='grayscale',
    class_mode='raw',
    batch_size=16,
    validate_filenames=True,
    shuffle=False
)

### 2- Loading models

In [ ]:
densenet201_model = load_model(
    "../trained_models/densenet201.h5",
    custom_objects={
        "binary_crossentropy_weighted_loss": set_binary_crossentropy_weighted_loss
    }
)

inceptionresnetv2_model = load_model(
    "../trained_models/inceptionresnetv2.h5",
    custom_objects={
        "binary_crossentropy_weighted_loss": set_binary_crossentropy_weighted_loss
    }
)

resnet152_model = load_model(
    "../trained_models/resnet152.h5",
    custom_objects={
        "binary_crossentropy_weighted_loss": set_binary_crossentropy_weighted_loss
    }
)

ensemble_model = load_model(
    "../trained_models/ensemble-model.h5",
    custom_objects={
        "binary_crossentropy_weighted_loss": set_binary_crossentropy_weighted_loss
    }
)

### 3- Predictions

In [ ]:
predictions_densenet201_model = densenet201_model.predict(
    x=test_datagenerator,
    steps=len(test_datagenerator),
    workers=1,
    verbose=1
)

# "Sigmoid" activation for output probabilities
predictions_densenet201_model = (predictions_densenet201_model >= 0.5).astype(np.float32)

In [ ]:
predictions_inceptionresnetv2_model = inceptionresnetv2_model.predict(
    x=test_datagenerator,
    steps=len(test_datagenerator),
    workers=1,
    verbose=1
)

# "Sigmoid" activation for output probabilities
predictions_inceptionresnetv2_model = (predictions_inceptionresnetv2_model >= 0.5).astype(np.float32)

In [ ]:
predictions_resnet152_model = resnet152_model.predict(
    x=test_datagenerator,
    steps=len(test_datagenerator),
    workers=1,
    verbose=1
)

# "Sigmoid" activation for output probabilities
predictions_resnet152_model = (predictions_resnet152_model >= 0.5).astype(np.float32)

In [ ]:
predictions_ensemble_model = ensemble_model.predict(
    x=test_datagenerator,
    steps=len(test_datagenerator),
    workers=1,
    verbose=1
)

# "Sigmoid" activation for output probabilities
predictions_ensemble_model = (predictions_ensemble_model >= 0.5).astype(np.float32)

### All CheXpert labels (all 14 labels)

#### 1- DenseNet-201 model

In [ ]:
report_densenet201_model = classification_report(
    y_true=test_datagenerator.labels.astype(np.uint8),
    y_pred=predictions_densenet201_model.astype(np.uint8),
    digits=4,
    target_names=y_cols
)

print(report_densenet201_model)

cm_densenet201 = multilabel_confusion_matrix(
    y_true=test_datagenerator.labels.astype(np.uint8),
    y_pred=predictions_densenet201_model.astype(np.uint8)
)

print(cm_densenet201)

#### 2- Inception-ResNet-V2 model

In [ ]:
report_inceptionresnetv2_model = classification_report(
    y_true=test_datagenerator.labels.astype(np.uint8),
    y_pred=predictions_inceptionresnetv2_model.astype(np.uint8),
    digits=4,
    target_names=y_cols
)

print(report_inceptionresnetv2_model)

cm_inceptionresnetv2 = multilabel_confusion_matrix(
    y_true=test_datagenerator.labels.astype(np.uint8),
    y_pred=predictions_inceptionresnetv2_model.astype(np.uint8)
)

print(cm_inceptionresnetv2)

#### 3- ResNet-152 model

In [ ]:
report_resnet152_model = classification_report(
    y_true=test_datagenerator.labels.astype(np.uint8),
    y_pred=predictions_resnet152_model.astype(np.uint8),
    digits=4,
    target_names=y_cols
)

print(report_resnet152_model)

cm_resnet152 = multilabel_confusion_matrix(
    y_true=test_datagenerator.labels.astype(np.uint8),
    y_pred=predictions_resnet152_model.astype(np.uint8)
)

print(cm_resnet152)

#### 4- Ensemble model

In [ ]:
report_ensemble_model = classification_report(
    y_true=test_datagenerator.labels.astype(np.uint8),
    y_pred=predictions_ensemble_model.astype(np.uint8),
    digits=4,
    target_names=y_cols
)

print(report_ensemble_model)

cm_ensemble = multilabel_confusion_matrix(
    y_true=test_datagenerator.labels.astype(np.uint8),
    y_pred=predictions_ensemble_model.astype(np.uint8)
)

print(cm_ensemble)

### CheXpert competition labels (first 5 labels only)

#### 1- DenseNet-201 model

In [ ]:
report_densenet201_model = classification_report(
    y_true=test_datagenerator.labels[:, :5].astype(np.uint8),
    y_pred=predictions_densenet201_model[:, :5].astype(np.uint8),
    digits=4,
    target_names=y_cols[:5]
)

print(report_densenet201_model)

cm_densenet201 = multilabel_confusion_matrix(
    y_true=test_datagenerator.labels[:, :5].astype(np.uint8),
    y_pred=predictions_densenet201_model[:, :5].astype(np.uint8)
)

print(cm_densenet201)

#### 2- Inception-ResNet-V2 model

In [ ]:
report_inceptionresnetv2_model = classification_report(
    y_true=test_datagenerator.labels[:, :5].astype(np.uint8),
    y_pred=predictions_inceptionresnetv2_model[:, :5].astype(np.uint8),
    digits=4,
    target_names=y_cols[:5]
)

print(report_inceptionresnetv2_model)

cm_inceptionresnetv2 = multilabel_confusion_matrix(
    y_true=test_datagenerator.labels[:, :5].astype(np.uint8),
    y_pred=predictions_inceptionresnetv2_model[:, :5].astype(np.uint8)
)

print(cm_inceptionresnetv2)

#### 3- ResNet-152 model

In [ ]:
report_resnet152_model = classification_report(
    y_true=test_datagenerator.labels[:, :5].astype(np.uint8),
    y_pred=predictions_resnet152_model[:, :5].astype(np.uint8),
    digits=4,
    target_names=y_cols[:5]
)

print(report_resnet152_model)

cm_resnet152 = multilabel_confusion_matrix(
    y_true=test_datagenerator.labels[:, :5].astype(np.uint8),
    y_pred=predictions_resnet152_model[:, :5].astype(np.uint8)
)

print(cm_resnet152)

#### 4- Ensemble model

In [ ]:
report_ensemble_model = classification_report(
    y_true=test_datagenerator.labels[:, :5].astype(np.uint8),
    y_pred=predictions_ensemble_model[:, :5].astype(np.uint8),
    digits=4,
    target_names=y_cols[:5]
)

print(report_ensemble_model)

cm_ensemble = multilabel_confusion_matrix(
    y_true=test_datagenerator.labels[:, :5].astype(np.uint8),
    y_pred=predictions_ensemble_model[:, :5].astype(np.uint8)
)

print(cm_ensemble)